### _Nothing in Biology Makes Sense Except in the Light of Evolution_ 
###### Theodosius  Dobzhansky

# TP-1: alineamiento global con Needleman Wunsch

Escriba una implementación en Python del algoritmo de alineamiento global de Needleman-Wunsch, 
dentro de las 4 celdas indicadas más abajo.

El alineamiento entre las 2 secuencias de `ab.fasta` debe tener un score de 158 y
el alineamiento entre las 2 secuencias de `ae.fasta` debe tener un score de 312.

--------

In [1]:
import numpy as np
from Bio import pairwise2, SeqIO
from enum import Enum

## Needleman-Wunsch con BioPython

In [2]:
aln = pairwise2.align.globalxx("TATA", "TCT")
aln[0].score

2.0

In [3]:
print(aln[0].seqA)
print(aln[0].seqB)

TA-TA
T-CT-


In [4]:
f_aln = SeqIO.parse("ab.fasta", 'fasta')
fas_A = next(f_aln)
fas_B = next(f_aln)

seq_A = str(fas_A.seq)
seq_B = str(fas_B.seq)

aln = pairwise2.align.globalxx(seq_A, seq_B)
aln[0].score

158.0

In [5]:
print(aln[0].seqA[300:])
print(aln[0].seqB[300:])

CGTCA-AAGATCTC-CAA-CCCGGGGACAGAGT-ACTGGCCGCGGATT-ACGACGGAA--ACCCGGTTTATACCGACTTCATCATGTTCAA-
C-TC-CAA-AT-T-ACAATCCC---GAC--A-TTA-T-----C---TTTA--A-GG-ATGA---GG-----A--GA----A-CA----C--G


In [6]:
f_aln = SeqIO.parse("ae.fasta", 'fasta')
fas_A = next(f_aln)
fas_E = next(f_aln)

seq_A = str(fas_A.seq)
seq_E = str(fas_E.seq)

aln = pairwise2.align.globalxx(seq_A, seq_E)
aln[0].score

312.0

In [7]:
print(aln[0].seqA[300:])
print(aln[0].seqB[300:])

GGCCCG-TCAAAGATC-TCCAACCCGGG-GACAGAG-TAC-TGGCC-GCG-GATT-ACGA-CGGAAACCCGG-TTTAT-ACCGACTTCATCATGTTCAA
GG--CGGTCAAAGA-CCTCCAACCC-GGCGACAG-GGT-CCTGG-CAGC-AGA-TGA-GAACGGAAACCC-GATTTA-CACCGAC-TC--C---TT---


------

## Needleman-Wunsch propio

#### Cosas q pueden ser útiles

### 1
`op` es un type que representa las 3 operaciones posibles en un alineamiento.

In [8]:
class op(Enum):
    GAP_A = 1
    GAP_B = 2
    MA_MM = 3

### 2
`score_mtx` contiene los scores de match y mismatch entre las distintas bases.
Para reproducir el comportamiento de la función`pairwise2.align.globalxx()` de biopython
asignaremos, por ahora, un score de `1` a cualquier match y `0` a cualquier otro match.

El gap score irá por fuera de `score_mtx` y será, circunstancialmente, de `0`.

`score_mtx` puede ser una lista de listas, `numpy.darray`, diccionario, o lo que crea conveniente.

In [9]:
import numpy as np
score_mtx = np.repeat(0, (4) * (4)).reshape(4, 4)
for i in range(0,4):
    score_mtx[i][i]=1
score_mtx

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

### 3
`traceback()` toma la tabla donde `nw()` memoizó los subproblemas (`tabla_sol`),
las 2 secuencias originales (`seqA` y `seqB`) y entrega las 2 secuencias modificadas (`aln_a` y `aln_b`),
que puestas una arriba de la otra, mostrarían un alineamiento correcto.

In [10]:
def traceback(scoreTable, firstSequence, secondSequence):
    firstIndex = 0
    secondIndex = 0
    
    steps , path = shortestPath(scoreTable, firstIndex,secondIndex)
    
    path.reverse() #The steps are saved backward, so I reverse them

    return align(firstSequence, secondSequence,path,0, 0, 0)
    
def align(firstSequence,secondSequence,operations,operationsIndex,firstSequenceIndex,secondSequenceIndex):
    firstSequenceSize = len(firstSequence)
    secondSequenceSize = len(secondSequence)

    if (firstSequenceIndex == firstSequenceSize and secondSequenceIndex == secondSequenceSize):
        return '', ''

    if firstSequenceIndex == firstSequenceSize:
        return '', secondSequence[secondSequenceIndex:]

    if secondSequenceIndex == secondSequenceSize:
        return '', firstSequence[firstSequenceIndex:]

    i = operations[operationsIndex]

    if i == 'Match':
        aln_a = firstSequence[firstSequenceIndex]
        aln_b = secondSequence[secondSequenceIndex]
        aln_a_res, aln_b_res = align(firstSequence,secondSequence,operations,operationsIndex + 1,firstSequenceIndex + 1,secondSequenceIndex + 1)
        return aln_a + aln_a_res, aln_b + aln_b_res

    elif i == 'gap_A':
        aln_a = '-'
        aln_b = secondSequence[secondSequenceIndex] 
        aln_a_res, aln_b_res = align(firstSequence,secondSequence,operations,operationsIndex + 1,firstSequenceIndex,secondSequenceIndex + 1)
        return aln_a + aln_a_res, aln_b + aln_b_res

    elif i == 'gap_B':
        aln_a = firstSequence[firstSequenceIndex] 
        aln_b = '-'
        aln_a_res, aln_b_res = align(firstSequence,secondSequence,operations,operationsIndex + 1,firstSequenceIndex + 1,secondSequenceIndex)
        return aln_a + aln_a_res, aln_b + aln_b_res
            
            
def shortestPath(scoreTable, firstIndex,secondIndex):
    rows, columns = scoreTable.shape
    if(firstIndex == rows and secondIndex == columns):
        return 0

    if(firstIndex == rows - 1):
        return columns-secondIndex - 1, [ 'gap_A' for firstIndex in range(columns - 1 - secondIndex)]

    if(secondIndex == columns - 1):
        return rows-firstIndex - 1,[ 'gap_B' for firstIndex in range(rows - 1 - firstIndex)]

    if scoreTable[firstIndex][secondIndex] == scoreTable[firstIndex + 1][secondIndex + 1]:
        steps1, firstPath   = shortestPath(scoreTable,firstIndex + 1,secondIndex)
        steps2, secondPath = shortestPath(scoreTable,firstIndex,secondIndex + 1)
        if steps1 < steps2:
            return steps1 + 1, firstPath + ['gap_A']
        return steps2 + 1, secondPath + ['gap_B']

    m, n = shortestPath(scoreTable,firstIndex + 1,secondIndex + 1)

    return m + 1, n + ['Match']

### 4
`nw()` implementa el algoritmo de needleman-wunsch. Toma las 2 secuencias en formato de `str` (`seqA_str` y `seqB_str`),
para igualar el comportamiento de `pairwise2.align.globalxx()`, la `score_mtx` y un `gap_score`.
Debe llamar a `traceback()` para luego devolver las 2 secuencias alineadas y el score del alineamiento (último elemento de `tabla_score`)

In [13]:
import numpy as np

def isMatch(aNucleotideValue):
    return aNucleotideValue == 1

def parseNucleotide(aNucleotide):
    values = {"T": 0,
               "C":1,
               "G": 2,
               "A":3 }
    return values[aNucleotide] 


def nw(firstSequence, secondSequence, score_mtx, gapScore = 0):

    fstSequenceSize = len(firstSequence)
    sndSequenceSize = len(secondSequence)
    scoreTable = np.repeat(0, (fstSequenceSize + 1) * (sndSequenceSize + 1)).reshape(fstSequenceSize + 1, sndSequenceSize + 1)
    
    for row in range(0, fstSequenceSize + 1):
        for column in range(0, sndSequenceSize + 1):

            actualValue = score_mtx[parseNucleotide(firstSequence[row - 1])][parseNucleotide(secondSequence[column - 1])]
            if isMatch(actualValue):
                scoreTable[row][column] = actualValue + scoreTable[row - 1][column - 1]
            else:
                scoreTable[row][column] = actualValue + max(
                                                  scoreTable[row-1][column] + actualValue, #gap in the first sequence  
                                                  scoreTable[row][column-1] + actualValue, #gap in the second sequence
                                                  scoreTable[row-1][column-1] - 1) #missmatch
                
            if row == 0:
                scoreTable[row][column] = 0
            if column == 0:
                scoreTable[row][column] = 0

    
    print(scoreTable)       
    firstSequenceAligned, secondSequenceAligned = traceback(scoreTable, firstSequence, secondSequence)                
    return firstSequenceAligned, secondSequenceAligned, scoreTable[fstSequenceSize, sndSequenceSize]

In [14]:
nw('TATA', 'TCT', score_mtx, 0)

[[0 0 0 0]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 2]
 [0 1 1 2]]


('TA-T', 'T-CTA', 2)

-------

## Pruebe su implementación

In [41]:
# Toma de input
f_aln = SeqIO.parse("ab.fasta", 'fasta')
fas_A = next(f_aln)
fas_B = next(f_aln)

seq_A = str(fas_A.seq)
seq_B = str(fas_B.seq)

In [42]:
# Confirme el score
aln_a, aln_b, score = nw(seq_A, seq_B, score_mtx)
score

[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   1   1   1]
 [  0   1   1 ...   2   2   2]
 ...
 [  0   1   2 ... 157 158 158]
 [  0   1   2 ... 157 158 158]
 [  0   1   2 ... 157 158 158]]


KeyboardInterrupt: 

In [ ]:
# Visualice el alineamiento. Recuerde que no es necesario que sea idéntico al de Biopython
str_aln_a = str().join(aln_a)
str_aln_b = str().join(aln_b)

print(str_aln_a[300:])
print(str_aln_b[300:])

In [ ]:
# Toma de input
f_aln = SeqIO.parse("ae.fasta", 'fasta')
fas_A = next(f_aln)
fas_E = next(f_aln)

seq_A = str(fas_A.seq)
seq_E = str(fas_E.seq)

In [ ]:
# Confirme el score
aln_a, aln_e, score = nw(seq_A, seq_E, score_mtx_1)
score

In [ ]:
# Visualice el alineamiento. Recuerde que no es necesario que sea idéntico al de Biopython
str_aln_a = str().join(aln_a)
str_aln_e = str().join(aln_e)

print(str_aln_a[300:])
print(str_aln_e[300:])